> ### 本文档介绍如何使用ipython进行并行计算

> 更详细的介绍，请见``ipyparallel``官方文档：https://ipyparallel.readthedocs.org/en/latest/

1. 配置
================
***

一般地，有两种方法启动ipython集群

### 1.1 基于命令行

    ipcluster start -n 8
    
上面的命令会在本地的机器上运行8个计算节点（engine）。

### 1.2 通过notebook配置

如果你使用ipython notebook，并且只是使用本机的话，那可以直接在web端进行配置：

<img src="img/jupyter_engin_config.png">

上面的页面，同样的，我们启动了8个计算节点。

In [ ]:
from ipyparallel import Client
rc = Client()

%px from ipyparallel import bind_kernel; bind_kernel()
%px %connect_info

2. 模拟函数
===============
***

我们这里将写一个函数，这个函数的基本特点：

* 具有一组待输入的参数；
* 函数单次运行比较消耗时间。

这样的函数可能代表了：

* 某个复杂模型的calibration；
* 或者单个策略的数年的回测；
* 一个计算量巨大的函数调用。

### 2.1 ``bsm_monte_carlo``

这里我们定义了一个基于Black - Scholes - Merton模型的Monte Carlo模拟函数，参数：

* ``s0``：当前价格
* ``strike``：行权价
* ``t``：到期时间
* ``r``：无风险利率
* ``sigma``：波动率
* ``num_paths``：Monte Carlo模拟次数

In [ ]:
import math
import numpy

def bsm_monte_carlo(s0, strike, t, r, sigma, num_paths):
    z = numpy.random.randn(num_paths)
    paths = s0 * numpy.exp((r - 0.5 * sigma ** 2) * t + sigma * math.sqrt(t) * z)
    pay_off =  numpy.maximum(paths - strike, 0.0)
    c0 = math.exp(-r * t) * sum(pay_off) / num_paths
    return c0

In [ ]:
print("Sample Price: %.4f" % bsm_monte_carlo(100., 105., 1., 0.05, 0.2, 50000))

下面的章节中，我们将以这个函数为基础，测试在ipython下并行计算的功效！

3. 串行 v.s. 并行
====================
***

对于Monte Carlo这类型的算法，经常要做的一件事就是测试算法的收敛性。这样的一份工作，无外乎要做以下的事：

* 多次运行上面的``bsm_monte_carlo``函数；
* 每次运行使用相同的``s0``, ``strike``, ``t``, ``r``以及``sigma``。但是``num_paths``逐次增加；
* 分析或者从图形直观分析收敛是否符合预期。

我们使用如下的``num_paths``的序列

```python
paths_set = range(5000, 500000, 5000)
```

In [ ]:
paths_set = range(5000, 500000, 5000)
from PyFin.Utilities import print_timing
from functools import partial

### 3.1 串行的做法

我们简单的一个接一个的，根据不同的参数，运行``bsm_monte_carlo``函数：

In [ ]:
@print_timing
def serial_run(paths_set):
    s0 = 100.
    strike = 105.
    t = 1.0
    r = 0.05
    sigma = 0.2
    
    part_func = partial(bsm_monte_carlo, s0, strike, t, r, sigma)
    return list(map(part_func, paths_set))

result = serial_run(paths_set)
print("Took: %.5s seconds" % result[0])

### 3.2 并行的做法

In [ ]:
import ipyparallel as ipp
from ipyparallel import require
rc = ipp.Client()
rc[:].push(dict(bsm_monte_carlo=bsm_monte_carlo))
lview = rc.load_balanced_view()
lview.block = True

使用``lview.parallel``decorator使得原先的函数可以运行于并行环境

In [ ]:
@lview.parallel()
@require("numpy", "math")
def bsm_monte_carlo_binded(num_paths):
    s0 = 100.
    strike = 105.
    t = 1.0
    r = 0.05
    sigma = 0.2
    return bsm_monte_carlo(s0, strike, t, r, sigma, num_paths)

In [ ]:
@print_timing
def parallel_run(paths_set):
    return bsm_monte_carlo_binded.map(paths_set)

result = parallel_run(paths_set)
print("Took: %.5s seconds" % result[0])

上面的例子，在8个计算节点的情况下，效能大概提高了3倍。

4. 并行运算的潜力
==================
***

并行本身是有一定的开销的，所以当本身计算量不是很大的时候，并行化本身的损耗可能大于计算函数消耗cpu的时间。在上面的例子中，如果``num_paths``选的不大的时候，``parallel_run``版本实际上要比``serial_run``版本更慢。

由此知道，大部分的时候，我们需要对并行化进行评估。主要回答两个问题：

* 在多大的运算规模之上，并行计算是有意义的？
* 并行化之后，我们可以获得的最大性能提升是多少倍？

下面的代码，基于这个玩具问题，对这两个问题进行了回答。最后我们以一张图，结束所有的讨论。

In [ ]:
max_nums_set = range(50000, 600000, 25000)

serial_result = [serial_run(range(5000, max_nums, 5000))[0] for max_nums in max_nums_set]
parallel_result = [parallel_run(range(5000, max_nums, 5000))[0] for max_nums in max_nums_set]
factors = numpy.array(parallel_result) / numpy.array(serial_result)

In [ ]:
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
from bokeh.models.formatters import NumeralTickFormatter

output_notebook()

TOOLS = ['box_zoom,box_select,resize,reset',HoverTool(tooltips=[
            ("Number of paths", "$x"),
            ("Time factor", "$y")],
            names=['scatter1', 'scatter2'])]

p = figure(title="Parallel monte carlo simulation on Cluster: with 8 engines", 
           plot_width=900, plot_height=600, tools=TOOLS)

p.line(max_nums_set, factors, legend="Parallel", line_color='green', line_dash=[5, 5],
       line_width=2.5, alpha=0.5)
p.line(max_nums_set, [1.] * len(factors), legend="Serial", line_color='red', 
       line_dash=[5, 5], line_width=2.5, alpha=0.5)
p.square(max_nums_set, factors, legend="Parallel", name='scatter1', color='green', size=6, alpha=0.5)
p.circle(max_nums_set, [1.] * len(factors), legend="Serial", name='scatter2', color='red', size=6, alpha=0.5)

p.xaxis.axis_label = 'number of paths'
p.yaxis.axis_label = 'Time factor (smaller is better)'
p.xaxis.formatter = NumeralTickFormatter(format='0 a')
p.yaxis.formatter = NumeralTickFormatter(format='0.00')

p.title_text_font = "times"
p.title_text_font_style = "italic"

p.legend.glyph_width = 100
p.legend.legend_spacing = 10

show(p)

### 4.1 结论

* 基本上在``num_paths``在10^5以上时，对这个问题，并行计算已经有意义了；

* 随着问题运算规模的上升，并行化的效率越来越高；

* 可以获取的最大性能提升大概在4倍左右。显著低于计算节点的数量（8）。这个是可以预见的，因为本人的计算机只有4个物理核心。